In [1]:
from pathlib import Path
import cv2
import numpy as np

In [3]:
video_input_path = 'inputs/media/Clips/'
videos = Path(video_input_path).rglob('*.mp4')
videos = [str(v) for v in videos]
# videos = [v for v in videos if 'msida' in v]
videos = [v for v in videos if 'original' in v.lower()]


In [5]:
# cv2 car counter by tracking
annotations_path = 'inputs/annotations/'
annotations = Path(annotations_path).rglob('*.txt')
annotations = [str(a) for a in annotations]
annotations = [a for a in annotations if 'train' in a.lower()]


In [6]:
# use annotations to do centroid tracking by splitting video in half
def count(video_path, annotation_path):
    video = cv2.VideoCapture(video_path)
    annotations = open(annotation_path, 'r')
    annotations = annotations.readlines()
    annotations = [a.strip() for a in annotations]
    # loop video frames
    frame_count = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break
        frame_count += 1
        if frame_count % 2 == 0:
            continue
        print(frame_count)
        # loop annotation
        for a in annotations:
            a = a.split(' ')
            a = [int(x) for x in a]
            x, y, w, h = a[0], a[1], a[2], a[3]
            x1, y1 = x, y
            x2, y2 = x + w, y + h
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, 'car', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('frame', frame)
            cv2.waitKey(1)


In [2]:
annotations_path = 'outputs/Yolo5_OGAnnotations/yolo5_classification/'
annotations = Path(annotations_path).rglob('*.txt')
annotations = [str(a) for a in annotations]
annotations = [a for a in annotations if 'test' in a.lower()]

In [ ]:
def fnid_centroid(x,y,w,h):
    x1, y1 = x, y
    x2, y2 = x + w, y + h
    return (x1 + x2) / 2, (y1 + y2) / 2

In [4]:
def centroid_distance(x1,y1,x2,y2):
    return np.sqrt((x1-x2)**2 + (y1-y2)**2)

In [ ]:
object_locations = {}
for a in annotations:
    annotations = open(a, 'r')
    annotations = annotations.readlines()
    annotations = [a.strip() for a in annotations]
    curr_centroids = []
    for a in annotations:
        a = a.split(' ')
        a = [int(x) for x in a]
        x, y, w, h = a[0], a[1], a[2], a[3]
        curr_centroids.append(fnid_centroid(x, y, w, h))

    # match new centroids with old
    for c in curr_centroids:
        if len(object_locations) == 0:
            object_locations[c] = c
        else:
            min_dist = 1000
            min_centroid = len(object_locations)
            for id, o in object_locations.items():
                dist = centroid_distance(o[0], o[1], c[0], c[1])
                if dist < min_dist:
                    min_dist = dist
                    min_centroid = o
            object_locations[min_centroid] = c

    # cv2 show centroids
    frame = cv2.imread(a.replace('txt', 'jpg'))
    for id, c in object_locations.items():
        cv2.circle(frame, (int(c[0]), int(c[1])), 5, (0, 0, 255), -1)
    cv2.imshow('frame', frame)
    cv2.waitKey(0)


